In [1]:
import torch
from rich.markdown import Markdown

device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
%cd ..

/home/oscarn/flan-gpt2


In [3]:
from models.model_utils import T5Model
# model_name = "OscarNav/flan-flan-t5-large-ft"
model_name = "google/flan-t5-xl"

wrapped_model = T5Model(model_name, device, quantization=True)
model = wrapped_model.get_model()
# model.resize_token_embeddings(len(wrapped_model.get_tokenizer()))
tokenizer = wrapped_model.get_tokenizer()

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-09-09 04:52:56,804 - accelerate.big_modeling - WARNING - You shouldn't move a model that is dispatched using accelerate hooks.


In [4]:
if torch.cuda.is_available():
    print(f"{torch.cuda.memory_allocated() / 1e9} GB allocated")

3.511022592 GB allocated


In [5]:
from eval.eval_utils import Evaluation
from data.data_utils import create_instruct_dataset


eval = Evaluation(model, tokenizer, device)

2025-09-09 04:52:57,137 - datasets - INFO - PyTorch version 2.5.1 available.


In [6]:
num_samples = 100
CAUSAL_LM = False

In [7]:
template = """
## INPUT PROMPT
{prompt}
## GROUND_TRUTH
{completion}
## MODEL OUTPUT
{prediction}
"""

### ANLI

In [9]:
dataset = create_instruct_dataset(num_samples, ["anli"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/16946 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16946 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.42it/s]


INPUT PROMPT                                                    

Paul Beard (4 August 1901 – 22 April 1989) was an English violinist, known particularly as leader of Sir Thomas    
Beecham's original London Philharmonic Orchestra and Sir Adrian Boult's BBC Symphony Orchestra. He was also a      
teacher, holding posts at the Royal Academy of Music and the Guildhall School of Music.                            

Based on the paragraph above can we conclude that "Paul Beard started as the leader of the London Philharmonic     
Orchestra"?                                                                                                        

OPTIONS:                                                                                                           

 • entailment                                                                                                      
 • neutral                                                                                                         
 • contradiction                                                                                                   


                                                   GROUND_TRUTH                                                    

neutral                                                                                                            


                                                   MODEL OUTPUT                                                    

neutral

## Common Gen

In [10]:
dataset = create_instruct_dataset(num_samples, ["common_gen"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:02<00:00,  2.04s/it]


INPUT PROMPT                                                    

Here are some concepts: ['fan', 'match', 'watch']                                                                  

What is a sentence about these concepts?                                                                           


                                                   GROUND_TRUTH                                                    

the fans watch the match                                                                                           


                                                   MODEL OUTPUT                                                    

fans watch match between football club and football player on television

## SQUAD

In [11]:
dataset = create_instruct_dataset(num_samples, ["squad"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
2025-09-09 04:53:43,947 - datasets.load - WARNING - Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /home/oscarn/.cache/huggingface/datasets/rajpurkar___squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Fri Nov 22 00:36:36 2024).
2025-09-09 04:53:43,953 - datasets.packaged_modules.cache.cache - WARNING - Found the latest cached dataset configuration 'plain_text' at /home/oscarn/.cache/huggingface/datasets/rajpurkar___squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Fri Nov 22 00:36:36 2024).


Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/87241 [00:00<?, ? examples/s]

Filter:   0%|          | 0/87241 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.35it/s]


INPUT PROMPT                                                    

University_of_Notre_Dame The College of Engineering was established in 1920, however, early courses in civil and   
mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the     
Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study – aerospace and  
mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer 
science and engineering, and electrical engineering – with eight B.S. degrees offered. Additionally, the college   
offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional    
B.A. and Master of Business Administration (MBA) degrees, respectively.                                            

Q: How many BS level degrees are offered in the College of Engineering at Notre Dame?                              


                                                   GROUND_TRUTH                                                    

eight                                                                                                              


                                                   MODEL OUTPUT                                                    

eight

## Cosmos QA

In [13]:
dataset = create_instruct_dataset(num_samples, ["cosmos_qa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the module from /home/oscarn/.cache/huggingface/modules/datasets_modules/datasets/allenai--cosmos_qa/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157 (last modified on Fri Nov 22 00:37:03 2024) since it couldn't be found locally at allenai/cosmos_qa, or remotely on the Hugging Face Hub.
2025-09-09 04:54:46,837 - datasets.load - WARNING - Using the latest cached version of the module from /home/oscarn/.cache/huggingface/modules/datasets_modules/datasets/allenai--cosmos_qa/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157 (last modified on Fri Nov 22 00:37:03 2024) since it couldn't be found locally at allenai/cosmos_qa, or remotely on the Hugging Face Hub.


Map:   0%|          | 0/25262 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25262 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:02<00:00,  2.69s/it]


INPUT PROMPT                                                    

Answer the question about text:                                                                                    

My feet were killing me at the end of the day as I was walking back and forth serving drinks and did n't get a     
moment to myself . I felt queasy to begin with but I just made sure I kept drinking water and tried to eat a       
sausage roll - ended up throwing half of it out . It was a good fast day , I just wish I had a good nights sleep   
Friday . The crowd were much better then the usual footy crowd except I got hit on a hell of a lot !               

Why am I feeling this way ? OPTIONS:                                                                               

 • I got food poisoning from the restaurant .                                                                      
 • I am overworked and need a good day 's rest .                                                                   
 • It was all in my head , I was actually fine .                                                                   
 • I was drinking way too much water .                                                                             


                                                   GROUND_TRUTH                                                    

I am overworked and need a good day 's rest .                                                                      


                                                   MODEL OUTPUT                                                    

I am overworked and need a good day's rest.

## CoQA

In [14]:
dataset = create_instruct_dataset(num_samples, ["coqa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=150, return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/7199 [00:00<?, ? examples/s]

Map:   0%|          | 0/4063 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4063 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:20<00:00, 20.01s/it]


INPUT PROMPT                                                    

Ibn Rushd (; 14 April 1126 – 10 December 1198), full name (), often Latinized as Averroes (), was a medieval       
Andalusian polymath. He wrote on logic, Aristotelian and Islamic philosophy, theology, the Maliki school of Islamic
jurisprudence, psychology, political and Andalusian classical music theory, geography, mathematics, and the        
mediæval sciences of medicine, astronomy, physics, and celestial mechanics. Ibn Rushd was born in Córdoba, Al      
Andalus (present-day Spain), and died at Marrakesh in present-day Morocco. His body was interred in his family tomb
at Córdoba. The 13th-century philosophical movement in Latin Christian and Jewish tradition based on Ibn Rushd's   
work is called Averroism.                                                                                          

Ibn Rushd was a defender of Aristotelian philosophy against Ash'ari theologians led by Al-Ghazali. Although highly 
regarded as a legal scholar of the Maliki school of Islamic law, Ibn Rushd's philosophical ideas were considered   
controversial in Ash'arite Muslim circles. Whereas al-Ghazali believed that any individual act of a natural        
phenomenon occurred only because God willed it to happen, Ibn Rushd insisted phenomena followed natural laws that  
God created.                                                                                                       

Ibn Rushd had a greater impact on Christian Europe, being known by the "the Commentator" for his detailed          
emendations to Aristotle. Latin translations of Ibn Rushd's work led the way to the popularization of Aristotle.   

Based on the article and the following list of answers, write a list of questions.                                 

  1 Córdoba, Al Andalus                                                                                            
  2 Spain                                                                                                          
  3 Marrakesh                                                                                                      
  4 Morocco                                                                                                        
  5 In his family tomb at Córdoba.                                                                                 
  6 A medieval Andalusian polymath                                                                                 
  7 logic                                                                                                          
  8 theology                                                                                                       
  9 geography                                                                                                      
 10 yes                                                                                                            
 11 14 April 1126                                                                                                  
 12 10 December 1198                                                                                               
 13 Aristotelian philosophy                                                                                        
 14 against Ash'ari theologians                                                                                    
 15 Al-Ghazali                                                                                                     
 16 "the Commentator"                                                                                              
 17 His emendations to Aristotle.                                                                                  
 18 yes                                                                                                            
 19 False.                                                                                            

## XSum

In [15]:
dataset = create_instruct_dataset(num_samples, ["xsum"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the dataset since EdinburghNLP/xsum couldn't be found on the Hugging Face Hub
2025-09-09 04:55:34,272 - datasets.load - WARNING - Using the latest cached version of the dataset since EdinburghNLP/xsum couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/oscarn/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/40db7604fedb616a9d2b0673d11838fa5be8451c (last modified on Mon Sep  8 19:18:01 2025).
2025-09-09 04:55:34,279 - datasets.packaged_modules.cache.cache - WARNING - Found the latest cached dataset configuration 'default' at /home/oscarn/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/40db7604fedb616a9d2b0673d11838fa5be8451c (last modified on Mon Sep  8 19:18:01 2025).


Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:04<00:00,  4.10s/it]


INPUT PROMPT                                                    

Summarize:                                                                                                         

Christie, 26, clocked a time of 42.565 seconds to finish ahead of Canadian Jamie Macdonald and Netherlands skater  
Yara van Kerkhof. The Nottingham-based Scot has been training with men and "learning to lose" to boost her medal   
prospects. She was controversially disqualified three times at Sochi 2014 but won World Cup and European honours   
last season. "It's been great to be back here in Calgary," said Christie. "I haven't had the most straightforward  
of competitions with a few falls on Friday and Saturday. My coaches and support team worked with me and my         
equipment overnight and the adjustments paid off." Christie was also part of the GB ladies relay team who broke the
3,000m British record on Friday. The team of Christie, captain Charlotte Gilmartin, Kathryn Thomson and Samantha   
Morrison finished in four minutes 13.719 seconds, with the previous record having stood at four minutes 14.57 secs.
Media playback is not supported on this device                                                                     


                                                   GROUND_TRUTH                                                    

Great Britain's Elise Christie won gold in the 500m at the speed skating World Cup in Calgary, Canada.             


                                                   MODEL OUTPUT                                                    

Great Britain's Jess Christie came fourth in the 10,000m in Calgary to continue her recovery from failing to retain
the Olympic title at 2016.

## BoolQ

In [17]:
dataset = create_instruct_dataset(num_samples, ["bool_q"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/9399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9399 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  1.47it/s]


INPUT PROMPT                                                    

It was recently revealed that prior to his abduction, Static teamed with Justice League member Black Lightning in  
order to stop former Blood Syndicate member Holocaust, who had tried to kill the superhero while he was acting as  
the keynote speaker at Ernest Hemingway High's senior graduation.                                                  

Can we conclude that are static shock and black lightning the same person?                                         

OPTIONS:                                                                                                           

 • True                                                                                                            
 • False                                                                                                           


                                                   GROUND_TRUTH                                                    

False                                                                                                              


                                                   MODEL OUTPUT                                                    

False

## Python Code

In [18]:
dataset = create_instruct_dataset(num_samples, ["python_code"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=1000, return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/16750 [00:00<?, ? examples/s]

Map:   0%|          | 0/16237 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16237 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:11<00:00, 11.44s/it]


INPUT PROMPT                                                    

Generate a Python script to sort the given list of words. ['bubble', 'selection', 'quicksort', 'insertion']        


                                                   GROUND_TRUTH                                                    

                                                                                                                   
 # function to sort the given list of words                                                                        
 def sort_words(words):                                                                                            
     # sort the list of words in alphabetical order                                                                
     words.sort()                                                                                                  
                                                                                                                   
     # print the sorted list of words                                                                              
     print(words)                                                                                                  
                                                                                                                   
 # driver code                                                                                                     
 words = ['bubble', 'selection', 'quicksort', 'insertion']                                                         
 sort_words(words)                                                                                                 
                                                                                                                   
 # output                                                                                                          
 ['bubble', 'insertion', 'quicksort', 'selection']                                                                 
                                                                                                                   


                                                   MODEL OUTPUT                                                    

s = input() t = s.split() l = ["bubble", "selection", "quicksort", "insertion"] t = t[:1] c = [t[i] for i in c] t =
[t.pop()] for i in e in c] c.sort()

## Eng-Spa

In [19]:
dataset = create_instruct_dataset(num_samples, ["eng_spa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.36s/it]


INPUT PROMPT                                                    

Eso me gustó. How do you say this sentence in English?                                                             


                                                   GROUND_TRUTH                                                    

I loved that.                                                                                                      


                                                   MODEL OUTPUT                                                    

That's how I liked it.

## PAWS

In [20]:
dataset = create_instruct_dataset(num_samples, ["paws"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Filter:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.45it/s]


INPUT PROMPT                                                    

Singer Angélique Kidjo and actor Djimon Hounsou were born in Cotonou , Benin .                                     

Singer Djimon Hounsou and actor Angélique Kidjo were born in Cotonou in Benin .                                    

Do the above sentences mean the same thing? OPTIONS:                                                               

 • No                                                                                                              
 • Yes                                                                                                             


                                                   GROUND_TRUTH                                                    

No                                                                                                                 


                                                   MODEL OUTPUT                                                    

No

## Quora

In [21]:
dataset = create_instruct_dataset(num_samples, ["quora"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/56402 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.47it/s]


INPUT PROMPT                                                    

What kind of food do people from the UK eat that is considered an acquired taste to the rest of the world?         


                                                   GROUND_TRUTH                                                    

crumpets, malt loaf, haggis, tripe, Christmas pudding, sprouts, just a few things                                  


                                                   MODEL OUTPUT                                                    

cheese

## Alpaca

In [23]:
dataset = create_instruct_dataset(num_samples, ["alpaca"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/20100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.47s/it]


INPUT PROMPT                                                    

Construct a phrase using the given words Pandemic, challenge, action                                               


                                                   GROUND_TRUTH                                                    

Our action in the face of the pandemic is the ultimate challenge.                                                  


                                                   MODEL OUTPUT                                                    

a challenge to action with pandemic